# Rendmi

Datadump used updated untl 06/07/2022

Source: http://dati.isprambiente.it/accesso-ai-dati/download/

In order to analyze tyhe dataset, gather all the data needed and turn it into a .json file we will use rdflib to query the graph.

In [3]:
# all imports
import pprint
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import XSD, RDFS, DCTERMS
from rdflib import Literal

Data had some errors that needed to be manualy corrected in order to maake the parser work.

**Specifically**:

Invalid lines sucH as *-schema#label>"[ sometext ]".*  were changed into *-schema#label> "[ sometext ]".* 

In [4]:
 # create an empty Graph for the rendmi dataset
g = rdflib.Graph()

# parse a local RDF file by specifying the format into the graph
result = g.parse("../datasets/originals/dissesto_07_06_2022.nt", format='nt')

WEe also downloaded the locations graph made available by ispra in order to get the region of provenance of each intervention and selecto only those happening in Emilia Romagna.

In [5]:
 # create an empty Graph for places dataset
places_data = rdflib.Graph()

# parse a local RDF file by specifying the format into the graph
result_places = places_data.parse("../datasets/originals/luoghi.nt", format='nt')

Looks abit weird as 50 entries are missing w.r.t. the original .nt file

## General Analysis

The first thing we wanna do is to undersatand the types of entries contained in this dataset.


Thus we previously gathered a [list](http://dati.isprambiente.it/sparql?default-graph-uri=&query=select+distinct+%3Fp+%3Fplabel%0D%0Awhere+%7B%0D%0A%3Fs+%3Fp+%3Fo.%0D%0A%3Fp+rdfs%3Alabel+%3Fplabel.%0D%0Afilter%28lang%28%3Fplabel%29+%3D+%27en%27%29%7D%0D%0A&format=text%2Fhtml&timeout=0&debug=on) of all predicates in the dataset and individuated http://www.w3.org/1999/02/22-rdf-syntax-ns#type as a startng point, as it is the predicate referring to the type of an entry

In [6]:
#look for types

types = set()

type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
instability = '<http://dati.isprambiente.it/ontology/core#Instability>'

for s, p, o in result.triples((None, type, None)):
    types.add(o)

print(types)

{rdflib.term.URIRef('http://purl.org/goodrelations/v1#BusinessEntity'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#LotStep'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#EconomicIndicator'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Repair'), rdflib.term.URIRef('http://purl.org/procurement/public-contracts#FrameworkAgreement'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Instability'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#AuthorityKind'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Contract'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#ConceptScheme'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Lithology'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#Concept'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Intervention')}


The result shows the main types of entities recorded in the graph.
At the moment the interesting fieldsfor us are Intervention, Instability and repair  interesting for us here. 

Lets see which predicateds are used with each one of them, keeping in mind weneed to find their geo loction, entity, etc. 

In [7]:
#Intervention

interventions = set()
type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
intervention = URIRef('http://dati.isprambiente.it/ontology/core#Intervention') 

for s, p, o in result.triples((None, type, intervention)):
    interventions.add(s)

print(list(interventions)[0])

http://dati.isprambiente.it/id/ihi/intervention/224/09


In [8]:
#predicates
preds_interventions = set()
intervention = URIRef('http://dati.isprambiente.it/id/ihi/intervention/232/02') 

for s, p, o in result.triples((intervention, None, None)):
    preds_interventions.add(p)

print(preds_interventions)

{rdflib.term.URIRef('http://purl.org/dc/terms/publisher'), rdflib.term.URIRef('http://purl.org/dc/terms/isPartOf'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#hasAgreement'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#primaryGeographicalFeature'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#officialInstabilityType'), rdflib.term.URIRef('http://purl.org/dc/terms/isReferencedBy'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://www.w3.org/2003/01/geo/wgs84_pos#location'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#contractingAuthority'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#amountFinanced')}


In [9]:
#test

for intervention_uri in interventions:
    intervention_this = URIRef(intervention_uri) 
    pred_this = set()

    for s, p, o in result.triples((intervention, None, None)):
        pred_this.add(p)
    
    if pred_this != preds_interventions:
        print(intervention_this, 'has different predicates')

In [10]:
#Instability

instabilities = set()
type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
instability = URIRef('http://dati.isprambiente.it/ontology/core#Instability') 

for s, p, o in result.triples((None, type, instability)):
    instabilities.add(s)

print(list(instabilities)[0])

http://dati.isprambiente.it/id/ihi/instability/183/0823di-5


In [11]:
# predicates
preds_instability = set()
instability = URIRef('http://dati.isprambiente.it/id/ihi/instability/108/082di-6') 

for s, p, o in result.triples((instability, None, None)):
    preds_instability.add(p)

print(preds_instability)

{rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#instabilityRelatedTo'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#instabilityType'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#instabilityGroup')}


In [12]:
#test

for intervention_uri in instabilities:
    instability_this = URIRef(intervention_uri) 
    preds_this = set()

    for s, p, o in result.triples((instability_this, None, None)):
        preds_this.add(p)
    
    if preds_this != preds_instability:
        print(instability_this, 'has different predicates')

In [13]:
#Repair

repairs = set()
type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
repair = URIRef('http://dati.isprambiente.it/ontology/core#Repair') 

for s, p, o in result.triples((None, type, repair)):
    repairs.add(s)

print(list(repairs)[0])

http://dati.isprambiente.it/id/ihi/repair/108/082si-33


In [14]:
 # predicates
preds_repairs = set()
repair = URIRef('http://dati.isprambiente.it/id/ihi/repair/036/061sf-32') 

for s, p, o in result.triples((repair, None, None)):
    preds_repairs.add(p)

print(preds_repairs)

{rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairType'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairRelatedTo')}


In [15]:
#test
x= 0
for repair_uri in repairs:
    repair_this = URIRef(repair_uri) 
    preds_thiss = set()

    for s, p, o in result.triples((repair_this, None, None)):
        preds_thiss.add(p)
    
    if preds_thiss != preds_repairs:
        x +=1
        # print(repair_this, 'has different predicates', preds_thiss)

print(x)
print(len(repairs))

4447
12600


**Test Results:**
4447 repair entries out of 12600 have an additional preposition, i.e., rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairCategory')}

## Interventions analysis

In [16]:
preds_interventions

{rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#amountFinanced'),
 rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#contractingAuthority'),
 rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#hasAgreement'),
 rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#officialInstabilityType'),
 rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#primaryGeographicalFeature'),
 rdflib.term.URIRef('http://purl.org/dc/terms/isPartOf'),
 rdflib.term.URIRef('http://purl.org/dc/terms/isReferencedBy'),
 rdflib.term.URIRef('http://purl.org/dc/terms/publisher'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
 rdflib.term.URIRef('http://www.w3.org/2003/01/geo/wgs84_pos#location')}

In [17]:
#find locality of interventions and 
er_interventions = set()
geo_feature = URIRef('http://dati.isprambiente.it/ontology/core#primaryGeographicalFeature')
parent1 = URIRef('http://www.geonames.org/ontology#parentADM1')
emro = URIRef('http://dati.isprambiente.it/id/place/emilia-romagna')

for interv in interventions:
    for s, p, o in result.triples((interv, geo_feature, None)):
        for place, p2, o2 in result_places.triples((o, parent1, None)):
          if o2 == emro:
            er_interventions.add(str(s))

print(len(er_interventions))



508


Now that we got all 508 interventions happening in Emilia-Romagna, we want to gather theier labels to make it more comprehensive 

In [32]:
intervention_locations = list()
location = URIRef('http://dati.isprambiente.it/ontology/core#primaryGeographicalFeature')
for inter in er_interventions:
    inter = URIRef(inter)
    for s, p, o in result.triples((inter, location, None)):
        this_label = str(o)

        intervention_locations.append(this_label)

print(len('tot interventions in Emilia_Romagna',intervention_locations))
intervention_locations = set(intervention_locations)
print(len('tot places in which interventions happen', intervention_locations))

508
192


In [40]:
luoghi_preds = set()


for s, p, o in result_places.triples((None, None, None)):
    luoghi_preds.add(p)

print(luoghi_preds)

{rdflib.term.URIRef('http://www.geonames.org/ontology#parentADM1'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#sameAs'), rdflib.term.URIRef('http://www.w3.org/2003/01/geo/wgs84_pos#long'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2003/01/geo/wgs84_pos#lat'), rdflib.term.URIRef('http://www.geonames.org/ontology#featureCode'), rdflib.term.URIRef('http://datiopen.istat.it/odi/ontologia/territorio/haCodIstat'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://www.geonames.org/ontology#featureClass'), rdflib.term.URIRef('http://www.geonames.org/ontology#name'), rdflib.term.URIRef('http://www.geonames.org/ontology#parentADM2'), rdflib.term.URIRef('http://www.opengis.net/ont/gml#polygon')}


In [41]:
names_places = dict()
label = URIRef('http://www.w3.org/2000/01/rdf-schema#label')
for urls in intervention_locations:
    place = URIRef(urls)
    for s, p, o in result_places.triples((place, label, None)):
        this_label = str(o)

        names_places[urls]=this_label

print(names_places)

{'http://dati.isprambiente.it/id/place/40032': 'Predappio', 'http://dati.isprambiente.it/id/place/37040': 'Monghidoro', 'http://dati.isprambiente.it/id/place/37037': 'Medicina', 'http://dati.isprambiente.it/id/place/35044': 'Viano', 'http://dati.isprambiente.it/id/place/33042': 'Sarmato', 'http://dati.isprambiente.it/id/place/37013': "Castel d'Aiano", 'http://dati.isprambiente.it/id/place/34034': 'Sissa', 'http://dati.isprambiente.it/id/place/36029': 'Palagano', 'http://dati.isprambiente.it/id/place/37047': 'Pianoro', 'http://dati.isprambiente.it/id/place/40012': 'Forlì', 'http://dati.isprambiente.it/id/place/99013': 'Riccione', 'http://dati.isprambiente.it/id/place/34046': 'Varsi', 'http://dati.isprambiente.it/id/place/40005': 'Castrocaro Terme e Terra del Sole', 'http://dati.isprambiente.it/id/place/99017': 'San Giovanni in Marignano', 'http://dati.isprambiente.it/id/place/37048': 'Pieve di Cento', 'http://dati.isprambiente.it/id/place/33005': 'Bobbio', 'http://dati.isprambiente.it/i

Right now we have the dictionary name_places containing key(place uri):value(rdfs label), we also decided to add latitude and longitude as we will need them fo maps visualizations and province code

In [97]:
cod_prov = {'Piacenza':'PC', 'Ferrara':'FE', 'Ravenna':'RA', 'Parma':'PR', 'Bologna':'BO', 'Rimini':'RN', "Forli'-Cesena":'FC', "Reggio nell'Emilia":'RE', 'Modena':'MO'}

In [101]:
x = set()
y = set()
province = URIRef('http://www.geonames.org/ontology#parentADM2')
for urls, place in names_places.items():
    # print(urls)
    url = URIRef(urls)
    
    for s, p, o in result_places.triples((url, province, None)):
        x.add(o)
        pred = URIRef('http://www.geonames.org/ontology#name')
        for s2, p2, o2 in result_places.triples((o,pred, None)):
            code = cod_prov[str(o2)]
            print(place, code)
            new_value = [place, code]
            names_places[urls]=new_value
print(names_places)





Predappio FC
Monghidoro BO
Medicina BO
Viano RE
Sarmato PC
Castel d'Aiano BO
Sissa PR
Palagano MO
Pianoro BO
Forlì FC
Riccione RN
Varsi PR
Castrocaro Terme e Terra del Sole FC
San Giovanni in Marignano RN
Pieve di Cento BO
Bobbio PC
Fontanelice BO
Castello d'Argile BO
San Clemente RN
San Felice sul Panaro MO
Lugo RA
Monterenzio BO
Vigolzone PC
Budrio BO
Vignola MO
Ravenna RA
Polinago MO
Alseno PC
Sasso Marconi BO
Montese MO
Maranello MO
Bondeno FE
Verghereto FC
Casola Valsenio RA
Castiglione dei Pepoli BO
Cortemaggiore PC
Calendasco PC
Medesano PR
Bentivoglio BO
Migliarino FE
Vetto RE
Vergato BO
Pennabilli RN
Castel San Giovanni PC
Faenza RA
Travo PC
Torriana RN
Langhirano PR
Gropparello PC
Bomporto MO
Grizzana Morandi BO
Mesola FE
Gaggio Montano BO
Ottone PC
Reggio nell'Emilia RE
Scandiano RE
Coriano RN
Collecchio PR
Toano RE
Lugagnano Val d'Arda PC
Castel Bolognese RA
Fidenza PR
Borghi FC
Brisighella RA
Verucchio RN
Tornolo PR
Lesignano de' Bagni PR
Ostellato FE
Terenzo PR
Modena MO



 rdflib.term.URIRef('http://www.w3.org/2003/01/geo/wgs84_pos#long'), 
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2003/01/geo/wgs84_pos#lat'), 

    rdflib.term.URIRef('http://www.geonames.org/ontology#name')